# Obtaining numerical solutions
## Ordinary differential equation for the infectious compartment
In developing `summer`, we have aimed to move away from notating our models in ODEs
and we will generally avoid ODE notation throughout this textbook.
This is because the `summer` API is intended to support code that is highly expressive,
such that the epidemiological intention can be easily gleaned from reading the code itself.
Therefore, there should be less need for an alternative form of notation.
Further, the ODEs used to notate epidemiological models
do not constitute the code used in simulation,
but rather the modellers intention for the construction of the system.
Therefore, there is considerable potential for the ODEs to diverge from the underlying code,
which is very common in our experience.

Nevertheless, for readers who are familiar with ODE notation,
we present the following expression for the infectious compartment of the model 
introduced in [notebook 02](./02-basic-model-intro.ipynb):
$$ \frac{dI(t)}{dt}=\frac{\beta S(t)}{N(t)}-\gamma I(t) $$
where the `infectious` and `susceptible` compartments 
are represented by $I$ and $S$ respectively,
and the contact rate and `recovery` rate
are represented by $\beta$ and $\gamma$ respectively.
Note that the division by $N(t)$ is handled behind the scenes by
_summer_ through the user's request for frequency-dependent transmission.

## Obtaining numerical solutions

This section presents some code that shows an approximation of what is happening inside the model we just built and ran.
This is just to show how we could have got the same solutions without using any solver at all,
and is intended to provide some insight into what is going on under the surface.
Clearly, coding models in this explicit way would make for overly verbose code
if we tried to create a complicated, highly stratified model in this way, 
and would not utilise many of the convenient features of the _summer_ platform.

In the example code below,
we use the Euler method to solve our compartmental system defined by the model's compartment structure and flows.
However, we don't typically use Euler in our modelling for policy, 
and a range of different solvers are available in the _summer_ backend.

In [ ]:
# If running on Google Colab, run the following line of code to install the summer package
# %pip install summerepi2

In [ ]:
import numpy as np
from jax import numpy as jnp
import pandas as pd
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, Function, Time

## Declare parameters
First, we'll declare a set of model specifications and parameters 
that we'll use with each of our two approaches.

In [ ]:
model_config = {
    "population": 1000.0,
    "seed": 10.0,
    "end_time": 20.0,
    "time_step": 1.0,
}
parameters = {
    "contact_rate": 1.0,
    "recovery_rate": 0.333,
}

## Explicit evaluation
The comments in the following cell work through how this explicit approach to model evaluation can be performed.

In [ ]:
# Get the evaluation times based on the requested parameters
time_period = model_config["end_time"]
num_steps = int(time_period / model_config["time_step"]) + 1
times = np.linspace(0.0, model_config["end_time"], num=num_steps)

# Prepare for outputs and populate the initial conditions
explicit_calcs = np.zeros((num_steps, 3))
explicit_calcs[0] = [
    model_config["population"] - model_config["seed"], 
    model_config["seed"],
    0.,
]

# Run the calculations at each modelled time step, except the first one
for t_idx, time in enumerate(times[1:], 1):

    # Get some quantities that we'll need later
    flow_rates = np.zeros(3)
    compartment_sizes = explicit_calcs[t_idx - 1]
    n_suscept = compartment_sizes[0]
    n_infect = compartment_sizes[1]
    n_pop = compartment_sizes.sum()
    
    # Apply the infection process under the assumption of frequency-dependent transmission
    force_of_infection = parameters["contact_rate"] * n_infect / n_pop
    infection_flow_rate = force_of_infection * n_suscept
    flow_rates[0] -= infection_flow_rate
    flow_rates[1] += infection_flow_rate

    # Recovery of the infectious compartment
    recovery_flow_rate = parameters["recovery_rate"] * n_infect
    flow_rates[1] -= recovery_flow_rate
    flow_rates[2] += recovery_flow_rate
    
    # Calculate compartment sizes at the next time step given the calculated flow rates
    explicit_calcs[t_idx] = compartment_sizes + flow_rates * model_config["time_step"]  
    
compartments = (
    "susceptible",
    "infectious",
    "recovered",
)
explicit_outputs_df = pd.DataFrame(explicit_calcs, columns=compartments, index=times)

## Comparison to the _summer_ approach
Now let's create the same system using a _summer_ compartmental model object.

In [ ]:
def get_sir_model(
    config: dict,
) -> CompartmentalModel:
    """
    This is the same model as from notebook 02
    (although we'll allow the compartments object previously
    declared to define the model's compartments here).
    
    Args:
        config: Values needed for model construction other than the parameter values   
    Returns:
        The summer model object
    """
    infectious_compartment = [
        "infectious",
    ]
    analysis_times = (0.0, config["end_time"])
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
    )
    pop = config["population"]
    seed = config["seed"]
    suscept_pop = pop - seed
    msg = "Seed larger than population"
    assert pop >= 0.0, msg
    model.set_initial_population(
        distribution={
            "susceptible": suscept_pop, 
            "infectious": seed}
    )
    
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"), 
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery_rate"), 
        source="infectious", 
        dest="recovered",
    )
    
    return model

## Plotting the two approaches
Last, let's confirm that the outputs are indeed the same with the two approaches.

In [ ]:
axis_labels = {"index": "time", "value": "compartment size"}
explicit_outputs_df.plot(labels=axis_labels)

In [ ]:
sir_model = get_sir_model(model_config)
sir_model.run(parameters=parameters, solver="euler")
compartment_values = sir_model.get_outputs_df()
compartment_values.plot(labels=axis_labels)

Here, we've set the solver of the compartmental model object to Euler,
which undertakes the same calculations as shown above.
In a sense, we are considering time as discrete-valued
and estimating the distribution of the population across compartments
at only a series of time points based on the transition rates
at the preceding time points.

Let's just confirm our visual impression 
that the results are the same numerically.

In [ ]:
diffs = explicit_outputs_df - compartment_values
diffs.max().max()

## Choice of solvers
In complex systems analysis,
there is extensive discussion of the relative merits
of various algorithms that can be used to solve
systems of ordinary differential equations.
Having shown the equivalence between 
the explicit solution and the Euler solver,
we should stress that we never use the Euler solver in practice.
The default solver used in the back-end of _summer_
is _scipy_'s _solve_ivp_ method.
This uses the fifth order Runge-Kutta algorithm
to obtain more accurate results by evaluating 
not only the first order rate of change, 
but other orders of derivatives at a range of time points.
Algorithms like this are more robust to "stiff" systems,
including ones that may have sudden fluctuations
in model conditions or parameters.

A detailed discussion of solver choice is outside 
the scope of this series.
However, let's take this to an extreme to illustrate
how the Euler method can break down.
As we've seen, the Euler method is essentially
assuming that the transition rate estimated at
a particular time point remains constant throughout the 
following time interval that we're estimating
the transition rate over.
Of course, this is never completely true.
For one thing, the sizes of the compartments 
do change somewhat because of the transitions themselves,
but may also change if the user has specified a particular
parameter should vary over time or in relation
to the emergent model state.
This can be addressed to some extent by
solving the system more frequently
(i.e. reducing the integration time step),
but there are more efficient ways to address this issue,
including the other solvers mentioned above.

To illustrate this, we'll artificially introduce a sudden fluctuation in
one of the model parameters (the contact rate)
that steps this parameter up to a very high value
for a short period of time,
and then drops it back to its baseline value.

In [ ]:
def step_and_go_back(
    time: Time, 
    start: Parameter, 
    duration: Parameter, 
    change_val: Parameter, 
    base_val: Parameter,
) -> callable:
    """
    Use numpy's where function to create a simple step function 
    that jumps up from it's base value to a higher value at a point in time
    and then drops back to the base value some period of time later.
    """
    offset = time - start
    return jnp.where(
        offset > 0.0, 
        jnp.where(offset < duration, change_val, base_val),
        base_val,
    )

def get_sir_spike_model(
    config: dict,
) -> CompartmentalModel:
    """
    Very similar model to the previous one,
    except uses the previous function to jump
    up the contact rate for a period of time.
    """
    infectious_compartment = compartments[1: 2]
    analysis_times = (0.0, config["end_time"])
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
    )
    pop = config["population"]
    seed = config["seed"]
    suscept_pop = pop - seed
    msg = "Seed larger than population"
    assert pop >= 0.0, msg
    model.set_initial_population(
        distribution={
            "susceptible": suscept_pop, 
            "infectious": seed}
    )
    
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Function(
            step_and_go_back, 
            [
                Time, 
                Parameter("contact_rate_step_start_time"),
                Parameter("contact_rate_step_duration"),
                Parameter("contact_rate_step_value"), 
                Parameter("contact_rate")
            ]
        ), 
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery_rate"), 
        source="infectious", 
        dest="recovered",
    )
    
    return model

In [ ]:
parameters.update(
    {
        "contact_rate_step_start_time": 10.0,
        "contact_rate_step_value": 10.0,
        "contact_rate_step_duration": 2.0,
    }
)

### Euler solver
The following cell shows that the Euler solver
copes poorly with this sudden change to parameter values.
This is because it calculates the rate of new infections at 
the point in time that the contact rate parameter jumps up
and effectively extrapolates the flow rate forward for the entire
time step it is evaluating.
In reality, the susceptible population would get rapidly
depleted within that time interval and transmission would slow.
This solver is unable account for this effect and
estimates that the susceptible population would become negative,
which is obviously impossible.

In [ ]:
sir_model = get_sir_spike_model(model_config)
sir_model.run(parameters=parameters, solver="euler")
compartment_values = sir_model.get_outputs_df()
compartment_values.plot(labels=axis_labels)

One approach to addressing this problem would be just 
to decrease the evaluation interval for the Euler solver
(or the explicit calculation approach).
If we evaluated the system every 0.1 time units,
we would get closer to the "true" result that we are seeking.
More generally, as the time step for evaluation approaches zero,
the simulation we obtain will get closer to this true result.
However, this will also increase the number of calculations that we need to make,
and in practice there are better ways to approach this problem.
### Runge-Kutta solver
_summer_'s default solver does a better job of simulating
what the true dynamics might be under this extreme parameter assumption,
estimating non-negative values for all compartments
over the simulation period.

In [ ]:
sir_model = get_sir_spike_model(model_config)
sir_model.run(parameters=parameters)
compartment_values = sir_model.get_outputs_df()
compartment_values.plot(labels=axis_labels)